In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By

import time, csv, os, environ

# Loading .env file
env = environ.Env()
env.read_env()

# lower is faster
WAITING_RATIO=0.5 

# CSV Writer
field_names = ['name', 'title', 'company','link']
def write_to_csv(data):    
    with open('ireland_directors.csv', 'a', newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = field_names)
        writer.writerows(data)


/Users/subodhk/Desktop/LinkedIn-Scrapping/venv/lib/python3.9/site-packages/environ/environ.py:639: UserWarning: /var/folders/t5/39m7s5ds4v7bw3xj7r71grgw0000gp/T/ipykernel_47102/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


In [5]:
# LinkedIn Sales Navigator Login

sales_url = "https://www.linkedin.com/sales/login"
opts = webdriver.ChromeOptions()
# opts.add_argument("--headless")     
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--window-size=1920x1080")
opts.add_argument("start-maximised")
browser= webdriver.Chrome('/Users/kanis/bin/chromedriver',options=opts)
browser.get(sales_url)

# Waiting for page to load
time.sleep(1)

# Switching to iframe
frame = browser.find_element(By.TAG_NAME, "iframe")
browser.switch_to.frame(frame)

# Entering Login Credentials
elem_username = browser.find_element("id","username")
elem_username.send_keys(os.environ['LINKEDIN_USERNAME'])
elem_psswd = browser.find_element("id","password")
elem_psswd.send_keys(os.environ['LINKEDIN_PASSWORD'])

# Submit login form
log_in_button=browser.find_element('xpath','//*[@type="submit"]')
log_in_button.click()

/var/folders/t5/39m7s5ds4v7bw3xj7r71grgw0000gp/T/ipykernel_47102/4214832483.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser= webdriver.Chrome('/Users/kanis/bin/chromedriver',options=opts)


In [6]:
input_url = input("Enter the url: ").strip()
print("Input URL: ", str(input_url))
browser.get(input_url)

Input URL:  https://www.linkedin.com/sales/search/people?savedSearchId=50568173


In [7]:

number_of_pages = int(input("Enter the number of pages you want to extract: "))

for i in range(number_of_pages):
    extracted_data = []
    time.sleep(5)
    start = time.time()
    initialScroll = 0
    finalScroll = 500
    while True:
        try:
            browser.execute_script(f"document.getElementById('search-results-container').scroll({initialScroll},{finalScroll})")
            initialScroll = finalScroll
            finalScroll += 500
            time.sleep(2*WAITING_RATIO)
            end = time.time()
            if round(end - start) > 15:
                print("End of scrolling")
                break
        except:
            print("Error in scrolling")

    src = browser.page_source
    soup_data = BeautifulSoup(src, 'lxml')
    ol_list = soup_data.find('ol', {'class': 'artdeco-list background-color-white _border-search-results_1igybl'})
    li_list = ol_list.find_all('li', {'class': 'artdeco-list__item pl3 pv3'})
    
    for item in li_list:
        name = item.find('span', {'data-anonymize': 'person-name'})
        title = item.find('span', {'data-anonymize': 'title'})
        company_name = item.find('a', {'class': 'ember-view t-black--light t-bold inline-block'})
        lead_url = item.find('a', {'class': 'ember-view'})
        try:
            data_item={}
            data_item["name"] = name.text.strip() if name else ""
            data_item["title"] = title.text.strip() if title else ""
            data_item["company"] = company_name.text.strip()  if company_name else ""
            data_item["link"]=  lead_url['href'] if lead_url else ""

            extracted_data.append(data_item)
        except Exception as e:
            print("Data not found for item: ", item)
            print("Error: ", e)
        
    write_to_csv(extracted_data)
    print(f"Page {i+1} done")

    next_button = browser.find_elements(By.CSS_SELECTOR,"[aria-label='Next']")
    if next_button[0].is_enabled():
        next_button[0].click()
    else:
        print("No more pages")
        break
    time.sleep(1*WAITING_RATIO)
    

End of scrolling
Page 0 done
End of scrolling
Page 1 done
End of scrolling
Page 2 done
End of scrolling
Page 3 done
